In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_parquet("hf://datasets/Process-Venue/Language_Indentification_v2/data/train-00000-of-00001.parquet")

df = df.dropna(subset=['Headline', 'Language'])

le = LabelEncoder()
df['label'] = le.fit_transform(df['Language'])

print(f"Number of samples: {len(df)}")
print(f"Languages: {list(le.classes_)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Number of samples: 95047
Languages: ['Assamese', 'Bengali', 'English', 'Gujarati', 'Hindi', 'Kannada', 'Kashmiri', 'Konkani', 'Malayalam', 'Marathi', 'Nepali', 'Odia', 'Punjabi', 'Sanskrit', 'Sindhi', 'Tamil', 'Telugu', 'Urdu']


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=5000)

X = vectorizer.fit_transform(df['Headline'])
y = df['label']

print(f"Feature matrix shape: {X.shape}")


Feature matrix shape: (95047, 5000)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=200, solver='saga', n_jobs=-1)
model.fit(X, y)

print("Model training complete.")


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Model training complete.


In [ ]:
def predict_language(text):
    x = vectorizer.transform([text])
    pred = model.predict(x)
    return le.inverse_transform(pred)[0]


In [ ]:
while True:
    text = input("Enter a sentence (or 'exit' to quit): ")
    if text.lower() == 'exit':
        print("Goodbye!")
        break
    language = predict_language(text)
    print(f"Predicted language: {language}\n")


Enter a sentence (or 'exit' to quit): வணக்கம்
Predicted language: Tamil

Enter a sentence (or 'exit' to quit): hello
Predicted language: English

Enter a sentence (or 'exit' to quit): নমস্কার
Predicted language: Bengali

Enter a sentence (or 'exit' to quit): నమస్కరం
Predicted language: Telugu

Enter a sentence (or 'exit' to quit): exit
Goodbye!
